## Creating Training Data

In [1]:
import cv2
import numpy as np

#Loading HAAR Cascade Classifier
face_classifier = cv2.CascadeClassifier('/Users/mac/Documents/Python Machine Learning/OpenCv/opencv-master/data/haarcascades/haarcascade_frontalface_default.xml')

def face_extractor(img):
    #Function detect the face and return the cropped face
    # It return input image if no face detected
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    #Cropping all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
        
    return cropped_face

cap = cv2.VideoCapture(0)
count = 0

#Collecting 100 samples of my face from webcam

while True:
    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200,200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        #saving file in directory
        file_name_path = './faces/user/'+str(count)+'.jpg'
        cv2.imwrite(file_name_path, face)
        
        #displaying live count on cam
        cv2.putText(face, str(count), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass
    
    if cv2.waitKey(1)==13 or count == 100:
        break

cap.release()
cv2.destroyAllWindows()
print("Collecting Samples Complete")

<h2> Training Model </h2>

In [2]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

#calling previously data that we made.
data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

Training_Data, Labels = [], []

for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

#crating numpy array both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialising facial recogniser
model = cv2.face.LBPHFaceRecognizer_create()

# Model Training
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model Trained Successfully")

Model Trained Successfully


<h2><p style = "color:Green";> Running Model </p></h2>

In [3]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('/Users/mac/Documents/Python Machine Learning/OpenCv/opencv-master/data/haarcascades/haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,255), 2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200,200))
    return img, roi

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    image, face = face_detector(frame)
    
    try :
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        # Passing face to prediction model
        results = model.predict(face)
        
        if results[1] < 500:
            confidence = int(100*(1-(results[1])/300))
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100,120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,50), 2)
        
        if confidence > 75:
            cv2.putText(image, "Unlocked", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Cropper', image)
            
        else :
            cv2.putText(image, "Locked", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Cropper', image)
    except:
        cv2.putText(image, "No Face Found", (220,120), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Locked", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Cropper', image)
        pass
    
    if cv2.waitKey(1)==13:
        break
    
cap.release()
cv2.destroyAllWindows()
        
    